In [ ]:
from forex.pre_training_data_prep.config import config
config

## Get previous Pandas dataframe and put it into Spark

In [ ]:
from forex.pre_training_data_prep.tasks.spark.task_convert_pandas_df_to_spark_df import task_convert_pandas_df_to_spark_df
task_convert_pandas_df_to_spark_df(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_conversion_to_spark'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
from forex.pre_training_data_prep.tasks.spark.array_tasks import task_pivot_and_sort_arrays
task_pivot_and_sort_arrays(**config)

In [ ]:
from forex.pre_training_data_prep.tasks.spark.array_tasks import task_diff_the_timestamp_arrays
task_diff_the_timestamp_arrays(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_pivot_and_sort'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
from pyspark.sql.types import ArrayType, IntegerType, FloatType

In [ ]:
def get_all_timestamps(timestamp_array, seconds_divisor):
    return [int(x) for x in range(min(timestamp_array), max(timestamp_array) + seconds_divisor, seconds_divisor)]

udf_get_all_timestamps = f.udf(get_all_timestamps, ArrayType(IntegerType()))

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = np.isnan(values_array)
#    has_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_nan_0_or_1)
#
#udf_do_nans_exist = f.udf(do_nans_exist, IntegerType())

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_non_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = ~np.isnan(values_array)
#    has_non_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_non_nan_0_or_1)
#    
#udf_do_non_nans_exist = f.udf(do_non_nans_exist, IntegerType())


def count_nans_in_array(values_array):
    values_array = np.array([np.array(values_array)])
    mask = np.isnan(values_array)
    nan_count = np.sum([int(x) for x in mask[0]])
    return int(nan_count)
    
udf_count_nans_in_array = f.udf(count_nans_in_array, IntegerType())



##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def forward_fill(values_array):
#    arr = np.array([values_array])
#    mask = np.isnan(arr)
#    idx = np.where(~mask, np.arange(mask.shape[1]), 0)
#    np.maximum.accumulate(idx, axis = 1, out = idx)
#    arr[mask] = arr[np.nonzero(mask)[0], idx[mask]]
#    to_return = list([float(x) for x in arr[0]])
#    return to_return
#
#udf_forward_fill = f.udf(forward_fill, ArrayType(FloatType()))


def locate_nans(timestamp_array, timestamp_all_array, values_array):

    # make sure we get an argsort in here later to ensure order of values is correct

    ts = np.array(timestamp_array, dtype = np.uint64) # ??
    ts_all = np.array(timestamp_all_array, dtype = np.uint64)  # we can probably make this smaller
    v = np.array(values_array, dtype = np.float64)  # we can probably make this smaller
    
    pdf = pd.DataFrame({'timestamp' : ts, 'values' : v})
    pdf_all = pd.DataFrame({'timestamp' : ts_all})

    pdf_joined = (
        pd.merge(
            pdf_all,
            pdf,
            on = 'timestamp',
            how = 'left',
        )
    )

    to_return = pdf_joined['values'].to_list()
    
    return to_return

udf_locate_nans = f.udf(locate_nans, ArrayType(FloatType()))

In [ ]:
import pyspark.sql.functions as f

from utilities.spark_session import get_spark_session

from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_all_timestamps
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_locate_nans
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_count_nans_in_array

def task_find_full_day_nans(**config):

    spark = get_spark_session(config['spark_config'])
    sdf_arrays = (
        spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
        .coalesce(config['n_processors_to_coalesce'])
        .orderBy('date_post_shift')
        .withColumn(
            'timestamps_all',
            udf_get_all_timestamps(f.col('sorted_timestamp_array'), f.lit(config['seconds_divisor']))
        )
    )
    
    for item in config['list_data_columns']:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_and_nans',
                udf_locate_nans(f.col('sorted_timestamp_array'), f.col('timestamps_all'), f.col('sorted_' + item + '_array'))
            )
        )

    for item in config['list_data_columns']:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_nan_count',
                udf_count_nans_in_array(f.col(item + '_and_nans'))
            )
        )

    sdf_arrays = sdf_arrays.withColumn('nan_count_full_day', f.col('return_nan_count'))
    for item in config['list_data_columns']:
        sdf_arrays = sdf_arrays.drop(item + '_nan_count')

    sdf_arrays.write.mode('overwrite').parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    spark.stop()

In [ ]:
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import task_find_full_day_nans
task_find_full_day_nans(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
sdf_arrays.show(5)
spark.stop()

In [ ]:
# stopped here for now

In [ ]:
## QA #1

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_qa = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
    .withColumn('has_nans', f.col('nan_count_full_day') > 0)
    .groupBy('has_nans').agg(f.count('date_post_shift').alias('number_of_days'))
)
sdf_qa.show()

In [ ]:
spark = get_spark_session(config['spark_config'])
sdf_qa_counts = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
)
sdf_qa_counts.show()

In [ ]:
import matplotlib.pyplot as plt
nan_counts_per_day = [x['nan_count_full_day'] for x in sdf_qa_counts.select('nan_count_full_day').collect()]

#plt.figure()
#plt.hist(nan_counts_per_day)
#plt.show()
#plt.close()

plt.figure()
plt.boxplot([nan_counts_per_day], widths=0.95)
plt.show()
plt.close()

In [ ]:
print(np.mean(nan_counts_per_day), np.median(nan_counts_per_day))

In [ ]:
pdf_qa_dates = sdf_qa_counts.toPandas()

In [ ]:
pdf_qa_dates['weekday'] = [x.weekday() for x in pdf_qa_dates['date_post_shift']]

plt.figure()
pdf_qa_dates.boxplot(by = 'weekday')
plt.show()
plt.close()

In [ ]:
## end QA #1

## Investigate consecutive NaNs for each day

By identifying the maximum number of NaNs in a row for a given day:

In [ ]:
import matplotlib.pyplot as plt

import pyspark.sql.functions as f

from forex.pre_training_data_prep.config import config
from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_max_consecutive_NaNs
from utilities.spark_session import get_spark_session

spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
for item in config['list_data_columns']:
    sdf_arrays = sdf_arrays.withColumn('max_consec_nan_' + item, udf_get_max_consecutive_NaNs(f.col(item + '_and_nans')))

#
#
#

from itertools import groupby
from pyspark.sql.types import BooleanType

def test_all_equality(*a_list):
    g = groupby(a_list)
    return next(g, True) and not next(g, False)

udf_test_all_equality = f.udf(test_all_equality, BooleanType())


columns_to_test = [f.col('max_consec_nan_' + x) for x in config['list_data_columns']]
sdf_arrays = sdf_arrays.withColumn('is_consec_equal', udf_test_all_equality(*columns_to_test))

columns_to_select = ['date_post_shift', 'nan_count_full_day']
columns_to_select.extend(['max_consec_nan_' + x for x in config['list_data_columns']])
columns_to_select.append('is_consec_equal')

sdf_temp = sdf_arrays.select(*columns_to_select)

#sdf_temp.show(5)

# should all be true
sdf_agg = (
    sdf_temp.select('date_post_shift', 'is_consec_equal')
    .groupBy('is_consec_equal')
    .agg(f.count('date_post_shift').alias('count_column'))
)
#sdf_agg.show()


sdf_arrays = (
    sdf_arrays
    .withColumn('max_daily_consec_nans', f.col('max_consec_nan_' + config['list_data_columns'][0]))
    .drop('is_consec_equal')
    .drop('sorted_timestamp_array', 'diff_sorted_timestamp_array')
)

for item in config['list_data_columns']:
    sdf_arrays = (
        sdf_arrays
        .drop(
            'max_consec_nan_' + item,
            'sorted_' + item + '_array',
        )
    )


#
# length tests
#
sdf_arrays = sdf_arrays.withColumn('len_timestamps_all', f.array_size(f.col('timestamps_all')))
for item in config['list_data_columns']:
    sdf_arrays = sdf_arrays.withColumn('len_' + item, f.array_size(f.col(item + '_and_nans')))

columns_to_test = [f.col('len_' + x) for x in config['list_data_columns']]
columns_to_test.extend(['len_timestamps_all'])

sdf_arrays = sdf_arrays.withColumn('is_length_equal', udf_test_all_equality(*columns_to_test))

# True
sdf_agg = (
    sdf_arrays.select('date_post_shift', 'is_length_equal')
    .groupBy('is_length_equal')
    .agg(f.count('date_post_shift').alias('count_column'))
)
#sdf_agg.show()

sdf_arrays = sdf_arrays.drop('is_length_equal', 'len_timestamps_all')
for item in config['list_data_columns']:
    sdf_arrays = sdf_arrays.drop('len_' + item)



sdf_arrays.show(3)

sdf_arrays.drop('nan_count_full_day', 'max_daily_consec_nans').write.mode('overwrite').parquet(config['directory_output'] + '/' + 'PLACEHOLDER.parquet')

spark.stop()                                       

## Trig functions

In [ ]:
from forex.pre_training_data_prep.tasks.spark.trig import add_trig
add_trig(**config)

## Window

#### Ensure there is room

In [ ]:
from forex.pre_training_data_prep.tasks.spark.spark_sliding_window import test_window_space
test_window_space(**config)

#### Compute the sliding windows

In [ ]:
from forex.pre_training_data_prep.tasks.spark.spark_sliding_window import do_sliding_window
do_sliding_window(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_sliding_window'])
sdf_arrays.show(3)
spark.stop()

In [ ]:
from forex.pre_training_data_prep.config import config
config

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_sliding_window_space_check'])
sdf_arrays.show(3)

In [ ]:
from utilities.spark_session import get_spark_session
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_sliding_window'])
sdf_arrays.show(3)

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, FloatType

def testI(a_list_of_lists):
    i = a_list_of_lists[0][0]
    return i

def testJ(a_list_of_lists):
    j = a_list_of_lists[20][0]
    return j

udf_test_i = f.udf(testI, IntegerType())
udf_test_j = f.udf(testJ, IntegerType())

udf_test_float_i = f.udf(testI, FloatType())
udf_test_float_j = f.udf(testJ, FloatType())

In [ ]:
(
    sdf_arrays
    .withColumn('I', udf_test_i(f.col('sw_timestamp')))
    .withColumn('J', udf_test_j(f.col('sw_timestamp')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

In [ ]:
(
    sdf_arrays
    .withColumn('I', udf_test_float_i(f.col('sw_return')))
    .withColumn('J', udf_test_float_j(f.col('sw_return')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

In [ ]:
(
    sdf_arrays
    .withColumn('I', udf_test_float_i(f.col('sw_sin_24')))
    .withColumn('J', udf_test_float_j(f.col('sw_sin_24')))
    .select('date_post_shift', 'I', 'J')
    .show(3)
)

## Expand arrays

In [1]:
from forex.pre_training_data_prep.config import config
config

{'database_name': 'django',
 'dag_id': 'NEW_prepare_forex_data',
 'list_data_columns': ['return', 'volatility', 'volume', 'lhc_mean'],
 'list_data_columns_no_scale': ['sin_24', 'cos_24'],
 'n_back': 180,
 'n_forward': 30,
 'offset': 1,
 'n_step': 10,
 'tz_name': 'US/Eastern',
 'price_type_name': 'mid',
 'instrument_name': 'EUR/USD',
 'interval_name': 'Minute',
 'retries_pull_forex_data': 1,
 'retry_delay_minutes_pull_forex_data': 5,
 'directory_output': '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning/forex/output',
 'filename_candlesticks_query_results': 'candlesticks_query_results.parquet',
 'filename_timezone_added': 'candlesticks_timezone_added.parquet',
 'filename_offset': 'candlesticks_timezone_weekday_offset.parquet',
 'filename_weekday_shift_merged': 'candlesticks_weekday_offset_merged.parquet',
 'filename_shift_days_and_hours_as_needed': 'candlesticks_shifted_as_needed.parquet',
 'filename_finalized_pandas': 'candlesticks_final

In [2]:
from forex.pre_training_data_prep.tasks.spark.array_expand import task_expand_arrays
task_expand_arrays(**config)

25/03/13 13:46:00 WARN Utils: Your hostname, emily-MS-7B96 resolves to a loopback address: 127.0.1.1; using 192.168.1.82 instead (on interface wlp5s0)
25/03/13 13:46:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 13:46:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_explode_array'])

In [4]:
sdf_arrays.count()        

523511

In [5]:
sdf_arrays.show(3)        

+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|date_post_shift| timestamp|              return|          volatility|              volume|            lhc_mean|              sin_24|              cos_24|
+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     2022-03-08|1646710980|[-1.7E-4, -8.0E-5...|[1.9E-4, 1.2E-4, ...|[86.0, 123.0, 115...|[1.08719, 1.08707...|[0.82658976, 0.82...|[0.56280494, 0.55...|
|     2022-03-08|1646711580|[2.0E-5, 0.0, -3....|[1.4E-4, 5.0E-5, ...|[80.0, 31.0, 116....|[1.0869033, 1.086...|[0.8503522, 0.852...|[0.52621394, 0.52...|
|     2022-03-08|1646712180|[3.0E-5, 0.0, -1....|[2.0E-4, 9.0E-5, ...|[81.0, 59.0, 48.0...|[1.0867567, 1.086...|[0.872496, 0.8746...|[0.48862123, 0.48...|
+---------------+----------+--------------------+--------------------+

## Find NaNs and decide what to do with them

In [6]:
from forex.pre_training_data_prep.config import config
config

{'database_name': 'django',
 'dag_id': 'NEW_prepare_forex_data',
 'list_data_columns': ['return', 'volatility', 'volume', 'lhc_mean'],
 'list_data_columns_no_scale': ['sin_24', 'cos_24'],
 'n_back': 180,
 'n_forward': 30,
 'offset': 1,
 'n_step': 10,
 'tz_name': 'US/Eastern',
 'price_type_name': 'mid',
 'instrument_name': 'EUR/USD',
 'interval_name': 'Minute',
 'retries_pull_forex_data': 1,
 'retry_delay_minutes_pull_forex_data': 5,
 'directory_output': '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning/forex/output',
 'filename_candlesticks_query_results': 'candlesticks_query_results.parquet',
 'filename_timezone_added': 'candlesticks_timezone_added.parquet',
 'filename_offset': 'candlesticks_timezone_weekday_offset.parquet',
 'filename_weekday_shift_merged': 'candlesticks_weekday_offset_merged.parquet',
 'filename_shift_days_and_hours_as_needed': 'candlesticks_shifted_as_needed.parquet',
 'filename_finalized_pandas': 'candlesticks_final

In [9]:
#from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_locate_nans
#from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_count_nans_in_array
# need something about first list item

In [20]:
import pyspark.sql.functions as f

from forex.pre_training_data_prep.tasks.spark.nan_related_tasks import udf_get_max_consecutive_NaNs
from utilities.spark_session import get_spark_session

def deal_with_post_sw_nans(**config):
    spark = get_spark_session(config['spark_config'])
    sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_explode_array'])

    items_to_check_list = config['list_data_columns'].copy()
    items_to_check_list.extend(config['list_data_columns_no_scale'])
    
    for item in items_to_check_list:
        sdf_arrays = sdf_arrays.withColumn('max_consec_nan_' + item, udf_get_max_consecutive_NaNs(f.col(item)))

    # We are skipping the QA equality test(s) here. Add it later.

    # is first item NaN?
    # drop if yes
    sdf_arrays = (
        sdf_arrays
        .withColumn('is_first_item_a_nan', f.isnan(f.col('return').getItem(0)))
        .where(~f.col('is_first_item_a_nan'))
        .drop('is_first_item_a_nan')
    )

    # max consecutive NaN count
    # decide on a threshold value after plotting the distribution
    sdf_arrays = (
        sdf_arrays
        .withColumn('max_consec_nans', f.col('max_consec_nan_return'))
        .drop(*['max_consec_nan_' + x for x in items_to_check_list])
    )

    



    #sdf_arrays.show(3)
    print(sdf_arrays.count())
    
    spark.stop()
    #sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_post_sw_nans'])

deal_with_post_sw_nans(**config)

508080
